In [ ]:
# Лабораторная по рекомендательным системам

In [2]:
import json

import pandas as pd
import numpy as np

from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt

%matplotlib inline

## 1. Подготовка данных

In [32]:
JSON_DATA_PATH = "Video_Games_5.json"
N = 10

def iter_json_data(path):
    with open(path) as f:
        for line in f:
            data = json.loads(line)
            yield data
            
def get_data_frame():
    uid_to_id = {}
    iid_to_id = {}
    
    cols = ["uid", "iid", "review", "rating", "dt"]
    rows = []
    for d in iter_json_data(JSON_DATA_PATH):
        uid = uid_to_id.setdefault(d["reviewerID"], len(uid_to_id))
        iid = iid_to_id.setdefault(d["asin"], len(iid_to_id))
        review = d["reviewText"]
        rating = float(d["overall"])
        dt = int(d["unixReviewTime"])
        rows.append((uid, iid, review, rating, dt))
        
    return pd.DataFrame(rows, columns=cols)

def split_df_by_dt(df, p=0.8):
    """Функция разбивает df на тестовую и тренировочную выборки по времени 
    публикации отзывов (значение времени в поле dt)
    
    :param p: персентиль значений dt, которые образуют тренировочную выборку. Например p=0.8 означает, что в 
    тренировочной части будут отзывы, соответствующие первым 80% временного интервала 
    :return: два pd.DataFrame объекта
    """
    border_dt = df.dt.quantile(p)
    #print("Min=%s, border=%s, max=%s" % (df.dt.min(), border_dt, df.dt.max()))
    training_df, test_df  = df[df.dt <= border_dt], df[df.dt > border_dt]
    #print("Размер до очистки:", training_df.shape, test_df.shape)
    # удаляем из тестовых данных строки, соответствующие пользователям или объектам, 
    # которых нет в тренировочных данных 
    # (пользователи - избегаем проблем для персональных систем, объекты - для всех)
    test_df = test_df[test_df.uid.isin(training_df.uid) & test_df.iid.isin(training_df.iid)]
    #print("Размер после очистки:", training_df.shape, test_df.shape)
    return training_df, test_df

In [4]:
df = get_data_frame()
training_df, test_df = split_df_by_dt(df)
del df

Min=939859200, border=1377129600.0, max=1405987200
Размер до очистки: (185427, 5) (46353, 5)
Размер после очистки: (185427, 5) (19174, 5)


## 2. Метрика качества и базовый класс модели

In [5]:
def hit_ratio(recs_dict, test_dict):
    """Функция считает метрику hit-ration для двух словарей
    :recs_dict: словарь рекомендаций типа {uid: {iid: score, ...}, ...}
    :test_dict: тестовый словарь типа {uid: {iid: score, ...}, ...}
    """
    hits = 0
    for uid in test_dict:
        if set(test_dict[uid].keys()).intersection(recs_dict.get(uid, {})):
            hits += 1
    return hits / len(test_dict)

def get_test_dict(test_df):
    """Функция, конвертирующая тестовый df в словарь
    """
    test_dict = {}
    for t in test_df.itertuples():
        test_dict.setdefault(t.uid, {})
        test_dict[t.uid][t.iid] = t.rating
    return test_dict

In [6]:
test_dict = get_test_dict(test_df)

Cоздадим базовые классы для рекомендателей, которые можно использовать при построении собственных моделей.

In [7]:
class BasicRecommender(object):
    def __init__(self):
        pass
    
    def get_recs(self, uid, top):
        """Строит рекомендации для пользователя uid
        :return: словарь типа {iid: score, ...}
        """
        return {}
    
    def get_batch_recs(self, uids, top):
        """Строит рекомендации для нескольких пользователей uids
        :return: словарь типа {uid: {iid: score, ...}, ...}
        """
        return {uid: self.get_recs(uid, top) for uid in uids}

## 3. Item-based collaborative filtering RS

Item-based CF основан на идее, что пользователь предпочтет объекты, похожие на те, что он приобретал ранее. Данные в CF модели представлены матрицей `user x item`, где ячейка матрицы соответствует рейтингу, который пользователь поставил объекту. Вместо рейтингов в матрице могут быть вероятности (т.е. вероятность, что пользователь воспользуется объектом). Для работы модели необходимо построить матрицу `item x item` схожести объектов. Обычно для построения матрицы схожести используется исходная матрица `user x item`. Чтобы уменьшить шумы в матрице схожести, для каждого объекта хранят только $K$ наиболее похожих объектов.

В простейшем случае рекомендации строятся путем нахождения объектов с наибольшим значением предсказанного рейтинга:
$$\hat{r}_{ui} = \frac{\sum_{j \in I_u} r_{uj} * sim(j, i)}{\sum_{j \in I_u} r_{uj}}$$

* $I_u$ - множество объектов, оцененных пользователем
* $sim(j, i)$ - схожесть между объектами $j$ и $i$

Часто из финальных рекомендаций для пользователя $u$ исключаются объекты $I_u$.

In [9]:
# вспомогательные функции, которые могут пригодиться при построении Item-based CF
def nullify_main_diagonal(m):
    positions = range(m.shape[0])
    eye = csr_matrix((np.ones(len(positions)), (positions, positions)), m.shape)
    return m - m.multiply(eye)


def get_topk(matrix, top, axis=1):
    """Converts source matrix to Top-K matrix
    where each row or column contains only top K values

    :param matrix: source matrix
    :param top: number of top items to be stored
    :param axis: 0 - top by column, 1 - top by row
    :return:
    """
    rows = []
    cols = []
    data = []

    if axis == 0:
        matrix = matrix.T.tocsr()

    for row_id, row in enumerate(matrix):
        if top is not None and row.nnz > top:
            top_args = np.argsort(row.data)[-top:]

            rows += [row_id] * top
            cols += row.indices[top_args].tolist()
            data += row.data[top_args].tolist()
        elif row.nnz > 0:
            rows += [row_id] * row.nnz
            cols += row.indices.tolist()
            data += row.data.tolist()

    topk_m = csr_matrix((data, (rows, cols)), (matrix.shape[0], matrix.shape[1]))

    if axis == 0:
        topk_m = topk_m.T.tocsr()

    return topk_m

In [24]:
from scipy.sparse import csr_matrix

def load_data(df):
    rows = []
    cols = []
    data = []
    
    uid_to_row = {}
    iid_to_col = {}
    
    for t in df.itertuples():
        row_id = uid_to_row.setdefault(t.uid, len(uid_to_row))
        col_id = iid_to_col.setdefault(t.iid, len(iid_to_col))
        rating = t.rating
        
        rows.append(row_id)
        cols.append(col_id)
        data.append(rating)
        
    ui_m = csr_matrix((data, (rows, cols)))
    return ui_m, uid_to_row, iid_to_col

In [48]:
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import binarize

class ItemBasedCollaborativeFilteringRS(object):
    def __init__(self):
        self.user_items = None
        self.items_similarity = None
        self.gamma = None
        self.uid_to_row = None
        self.iid_to_col = None
        self.col_to_iid = None
    
    def _calculate_z_score(self):
        item_avg_rating = self.user_items.mean(axis=0)
        
        user_items_norm = self.user_items - binarize(self.user_items).multiply(item_avg_rating)
        user_items_norm.data = user_items_norm.data ** 2

        user_items_norm_sum = user_items_norm.sum(axis=0)
        ratings_per_item = binarize(self.user_items).sum(axis=0)
        sigmas = np.sqrt(user_items_norm_sum / ratings_per_item)

        z_score = (self.user_items - binarize(self.user_items).multiply(item_avg_rating)).multiply(1 / sigmas)
        return z_score.tocsr()

    def create_recommendations(self, training_df, gamma):
        if self.user_items is None:
            self.user_items, self.uid_to_row, self.iid_to_col = load_data(training_df)
            self.col_to_iid = {v: k for k, v in self.iid_to_col.items()}
            self.gamma = gamma
        
        self.user_items = self._calculate_z_score()
        self.items_similarity = cosine_similarity(self.user_items.T.tocsr(), dense_output=False)
        self.items_similarity = nullify_main_diagonal(self.items_similarity)
        self.items_similarity = get_topk(self.items_similarity, top=30)
        self.items_similarity = normalize(self.items_similarity)
        
        intersection_cnt = (binarize(self.user_items.T) @ binarize(self.user_items)).tocsr()

        intersection_cnt.data /= self.gamma
        intersection_cnt.data[intersection_cnt.data > 1] = 1

        self.items_similarity = self.items_similarity.multiply(intersection_cnt)
        
        return self
    
    def recommend(self, uids):
        ans = {}
        uid_rows = [self.uid_to_row[uid] for uid in uids if uid in self.uid_to_row]
    
        recs = self.user_items[uid_rows] @ self.items_similarity.T
        for rec, uid in zip(recs, uids):
            elem = {}
            for arg_id in np.argsort(rec.data)[-10:][::-1]:
                col = rec.indices[arg_id]
                iid = self.col_to_iid[col]
                score = rec.data[arg_id]
                elem[iid] = score
            ans[uid] = elem
        return ans

In [44]:
def cross_validate(model, training_df, tries=4, params={}):
    cv_scores = []
    training_df_cv = training_df.copy()

    for i in range(tries):
        training_df_cv, test_df_cv = split_df_by_dt(training_df_cv)
        recommendations = model.create_recommendations(training_df_cv, **params).recommend(test_df_cv['uid'])
        score = hit_ratio(recommendations, get_test_dict(test_df_cv))
        cv_scores.append(score)

    return cv_scores

from sklearn.model_selection import ParameterGrid

def grid_search(model, training_df, parameters):
    best_score = 0
    for parameter in ParameterGrid(parameters):
        model_score = np.average(cross_validate(model, training_df, params=parameter))
        if model_score > best_score:
            best_score = model_score
            best_params = parameter
        
    return best_params

Подберем параметр гамма

In [50]:
model = ItemBasedCollaborativeFilteringRS()
params = {'gamma': range(10,110,10)}
print(grid_search(model, training_df, params))

{'gamma': 100}


Тренируем модель с заданным параметром

In [51]:
model = ItemBasedCollaborativeFilteringRS()
recs = model.create_recommendations(training_df, 100).recommend(test_df['uid'])

hit_ratio(ans, test_dict)

0.09024211298606016

оценка для Item-based collaborative filtering RS - __0.09024211298606016__